In [18]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import glob
import errno
import pandas
from sklearn import naive_bayes, metrics, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Laptop
[nltk_data]     Ku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def word_token(text):
    tokens = word_tokenize(text)

    return tokens

In [20]:
def casefolding(text):
    text = text.lower()
    return text

In [21]:
def stemming (tokens):
    porter_stemmer = PorterStemmer()
    result = list()
    for word in tokens:
        result.append(porter_stemmer.stem(word))
        
    return ' '.join(result)
    return text

In [22]:
def stopword (tokens):
    stop_words = set(stopwords.words('english'))
    clean_text = []
    for word in tokens:
        if word not in stop_words:
            clean_text.append(word)
    return clean_text

In [23]:
def noise_rem (text):
    text = re.sub('[^\w\s]','', text)
    return text

In [24]:
def masukdata (path,cl):
    files = glob.glob(path)
    labels, texts = [], []
    for review in files:
        f = open(review,'r')
        baca = f.read()
        f.close()
        
        baca = casefolding(baca)
        baca = noise_rem(baca)
        tok = word_token(baca)
        tok = stopword(tok)
        tok = stemming(tok)
        baca = ''.join(tok)
        
        labels.append(cl)
        texts.append(baca)
    trainDF = pandas.DataFrame()
    trainDF['text'] = texts
    trainDF['label'] = labels
    
    return trainDF

In [25]:
def kumpuldata(seg):
    trainData = pandas.DataFrame()
    trainData = masukdata('./'+seg+'/neg/*.txt','neg')
    trainData = trainData.append(masukdata('./'+seg+'/pos/*.txt','pos'), ignore_index=True)
    trainData.reset_index()
    return trainData

In [44]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_y, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [46]:
def mainKlas():
    sets = kumpuldata('train')
    train_x = sets['text']
    train_y = sets['label']
    sets = kumpuldata('test')
    valid_x = sets['text']
    valid_y = sets['label']
    
    encoder = preprocessing.LabelEncoder()
    train_y = encoder.fit_transform(train_y)
    valid_y = encoder.fit_transform(valid_y)
    
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    count_vect.fit(sets['text'])
    train_xc =  count_vect.transform(train_x)
    valid_xc =  count_vect.transform(valid_x)
    
    accuracy = train_model(naive_bayes.MultinomialNB(), train_xc, train_y, valid_xc, valid_y)
    print ("NB, Count Vectors: ", accuracy)
    #naive_bayes.MultinomialNB().fit(train_xc, train_y)
    #print(train_xc)
    #print(valid_xc)
    #predictions = naive_bayes.MultinomialNB().predict(valid_xc)
    
    #acc = metrics.accuracy_score(predictions, valid_y)
    #print (acc)
        
if __name__ == "__main__":
    mainKlas()

NB, Count Vectors:  0.73
